# "*Natural climate reconstruction in the Stave Churches through time series processing with variational autoencoders*"

**FFPython2**

Noemi Manara 2022909

Lorenzo Rosset 2021859

Francesco Zambelli 2029014

Andrea Zanola 2027588

In [ ]:
!git clone https://github.com/rossetl/Natural-climate-reconstruction-with-VAE.git

# **Part 2: Implementation of Variational Autoencoders for the time series reconstruction task**

In the following sections we are going to show some implementations of Variational Autouncoders built in order to reconstruct the hystorical climate of the Ringebu church.